In [1]:
from fromscratchtoml.neural_network.models import Sequential
from fromscratchtoml.neural_network.optimizers import StochasticGradientDescent, Adam
from fromscratchtoml.neural_network.layers import Dense, Activation

import numpy as np
from sklearn.model_selection import train_test_split

from fromscratchtoml.toolbox.random import Distribution
from fromscratchtoml.toolbox.preprocess import to_onehot

from fromscratchtoml.neural_network.layers import RNN

In [2]:
%%script false 
import csv
import nltk
nltk.download('punkt')
import itertools

vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

# Read the data and append SENTENCE_START and SENTENCE_END tokens
print ("Reading CSV file...")
with open('mldata/reddit-comments-2015-08.csv', 'r') as f:
#     reader = csv.reader(f, skipinitialspace=True)
    reader = csv.DictReader(f)
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x['body'].lower()) for x in reader])
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print ("Parsed %d sentences." % (len(sentences)))
    
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print ("Found %d unique words tokens." % len(word_freq.items()))

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print ("Using vocabulary size %d." % vocabulary_size)
print ("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print ("\nExample sentence: '%s'" % sentences[0])
print ("\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])

In [3]:
import pickle

# with open('company_data.pkl', 'wb') as output:
#     pickle.dump(X_train, output, pickle.HIGHEST_PROTOCOL)
#     pickle.dump(y_train, output, pickle.HIGHEST_PROTOCOL)

# del X_train
# del y_train

with open('company_data.pkl', 'rb') as input1:
    X_train = pickle.load(input1)
    y_train = pickle.load(input1)

In [4]:
%%script false 
import numpy as np
# Create the training data
X_train = np.array([np.array(([word_to_index[w] for w in sent[:-1]])) for sent in tokenized_sentences])
y_train = np.array([np.array(([word_to_index[w] for w in sent[1:]])) for sent in tokenized_sentences])

In [5]:
len(X_train[10])

45

In [6]:
X_train[0], len(X_train[0])

(array([   0,    6, 3495,    7,  155,  796,   25,  222,    8,   32,   20,
         202, 4954,  350,   91,    6,   66,  207,    5,    2]), 20)

In [7]:
vocabulary_size = 8000

In [8]:
rnn = RNN(vocab_size=vocabulary_size)

In [9]:
len(X_train)

79170

In [10]:
temp = []
for x in X_train[0]:
    t = np.zeros(8000)
    t[x] = 1
    temp.append(t)
temp = np.array(temp)

In [11]:
#rnn.forward(temp)
#rnn.forward(X_train[0])

In [12]:
temp.shape

(20, 8000)

In [13]:
model1 = Sequential(verbose=1, vis_each_epoch=True)
model1.add(RNN(vocab_size=vocabulary_size, seed=1, memory_window=4))
model1.add(Activation('sigmoid'))
sgd = StochasticGradientDescent(learning_rate=0.0005)
model1.compile(optimizer=sgd, loss="cross_entropy")

In [14]:
np.array([y_train[0]]).shape

(1, 20)

In [15]:
def to_onehot(X, reverse=False):
    temp = []
    for word in X:
        if reverse:
            print(np.argmax(word), len(word))
            temp.append(np.argmax(word))
        else:
            t = np.zeros(8000)
            t[word] = 1
            temp.append(t)
    temp = np.array(temp)
    return temp

In [16]:
to_onehot(y_train[0]).shape
# np.array([X_train[0]]).shape

(20, 8000)

In [17]:
#model1.fit(np.array([temp]), np.array([y_train[0]]), epochs=1)
model1.fit(X_train[:100], y_train[:100], epochs=1)
# model1.forwardpass(np.array(X_train[0]))

ValueError: shapes (1,) and (8000,100) not aligned: 1 (dim 0) != 8000 (dim 0)

In [ ]:
model1.predict(np.array([X_train[100]]), one_hot=0)

In [ ]:
X_train[1], y_train[1]

In [ ]:
model1.layers[0].W_hh

In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np

from mlfromscratch.utils import train_test_split, to_categorical, normalize, Plot
from fromscratchtoml.neural_network.models import Sequential
from fromscratchtoml.neural_network.optimizers import Adam
from fromscratchtoml.neural_network.layers import Dense, Activation

import numpy as np
from sklearn.model_selection import train_test_split

from fromscratchtoml.toolbox.random import Distribution
from fromscratchtoml.toolbox.preprocess import to_onehot

from fromscratchtoml.neural_network.layers import RNN

def main():

    optimizer = Adam()
#     optimizer = StochasticGradientDescent()

    def gen_mult_ser(nums):
        """ Method which generates multiplication series """
        X = np.zeros([nums, 10, 61], dtype=float)
        y = np.zeros([nums, 10, 61], dtype=float)
        for i in range(nums):
            start = np.random.randint(2, 7)
            mult_ser = np.linspace(start, start*10, num=10, dtype=int)
            X[i] = to_categorical(mult_ser, n_col=61)
            y[i] = np.roll(X[i], -1, axis=0)
        y[:, -1, 1] = 1 # Mark endpoint as 1
        return X, y


    def gen_num_seq(nums):
        """ Method which generates sequence of numbers """
        X = np.zeros([nums, 10, 20], dtype=float)
        y = np.zeros([nums, 10, 20], dtype=float)
        for i in range(nums):
            start = np.random.randint(0, 10)
            num_seq = np.arange(start, start+10)
            X[i] = to_categorical(num_seq, n_col=20)
            y[i] = np.roll(X[i], -1, axis=0)
        y[:, -1, 1] = 1 # Mark endpoint as 1
        return X, y

    X, y = gen_mult_ser(3000)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

    # Model definition
#     clf = NeuralNetwork(optimizer=optimizer,
#                         loss=CrossEntropy)
    clf = Sequential()
    clf.add(RNN(units=10, memory_window=5, vocab_size=61))
    clf.add(Activation('softmax'))
    clf.compile(optimizer=optimizer, loss='cross_entropy')
#     clf.summary("RNN")

    # Print a problem instance and the correct solution
    tmp_X = np.argmax(X_train[0], axis=1)
    tmp_y = np.argmax(y_train[0], axis=1)
    print ("Number Series Problem:")
    print ("X = [" + " ".join(tmp_X.astype("str")) + "]")
    print ("y = [" + " ".join(tmp_y.astype("str")) + "]")
    print ()

    train_err, _ = clf.fit(X_train, y_train, epochs=500, batch_size=512)

    # Predict labels of the test data
    y_pred = np.argmax(clf.predict(X_test), axis=2)
    y_test = np.argmax(y_test, axis=2)

    print ()
    print ("Results:")
    for i in range(5):
        # Print a problem instance and the correct solution
        tmp_X = np.argmax(X_test[i], axis=1)
        tmp_y1 = y_test[i]
        tmp_y2 = y_pred[i]
        print ("X      = [" + " ".join(tmp_X.astype("str")) + "]")
        print ("y_true = [" + " ".join(tmp_y1.astype("str")) + "]")
        print ("y_pred = [" + " ".join(tmp_y2.astype("str")) + "]")
        print ()
    
    accuracy = np.mean(accuracy_score(y_test, y_pred))
    print ("Accuracy:", accuracy)

    training = plt.plot(range(500), train_err, label="Training Error")
    plt.title("Error Plot")
    plt.ylabel('Training Error')
    plt.xlabel('Iterations')
    plt.show()

main()

/home/markroxor/Documents/jellAIfish/fromscratchtoml/env/lib/python3.6/site-packages/fromscratchtoml-0.0.2-py3.6.egg/fromscratchtoml/toolbox/__init__.py:14: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/markroxor/Documents/jellAIfish/fromscratchtoml/env/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/markroxor/Documents/jellAIfish/fromscratchtoml/env/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app

ImportError: cannot import name 'Adam'

In [11]:
import numpy as np
from fromscratchtoml.neural_network.models import Sequential
from fromscratchtoml.neural_network.layers import Dense, Activation
from fromscratchtoml.neural_network.optimizers import StochasticGradientDescent

from sklearn.model_selection import train_test_split
from fromscratchtoml.toolbox.preprocess import to_onehot
from fromscratchtoml.toolbox.random import Distribution



X11 = Distribution.radial_binary(pts=300,
               mean=[0, 0],
               st=1,
               ed=2, seed=20)
X22 = Distribution.radial_binary(pts=300,
               mean=[0, 0],
               st=4,
               ed=5, seed=10)

Y11 = np.ones(X11.shape[0])
Y22 = np.zeros(X11.shape[0])

X = np.vstack((X11, X22))
y = np.hstack((Y11, Y22))

y = to_onehot(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=50, random_state=42)
        
        
        
model = Sequential(verbose=0)

model.add(Dense(10, input_dim=2, seed=1))
model.add(Activation('sigmoid'))

model.add(Dense(2, seed=7))
model.add(Activation('tanh'))

model.add(Dense(2, seed=2))
model.add(Activation('relu'))

model.add(Dense(2, seed=3))
model.add(Activation('leaky_relu'))

model.add(Dense(2, seed=4))
model.add(Activation('linear'))

model.add(Dense(2, seed=6))
model.add(Activation('softmax'))


sgd = StochasticGradientDescent(learning_rate=0.05)
model.compile(optimizer=sgd, loss="mean_squared_error")

model.fit(X_train, y_train, epochs=14, batch_size=4)

expected_biases = np.array([[0.08650937, 1.00013189]], dtype=np.float128)
# self.assertTrue(np.allclose(expected_biases, model.layers[-1].biases))

expected_weights = np.array([[-0.49908263, -0.17316507], [-0.42623203, 0.48448988]], dtype=np.float128)
# self.assertTrue(np.allclose(expected_weights, model.layers[-1].weights))
model.accuracy(X_test, y_test)
model.layers[-2].biases, model.layers[-2].weights

[█████████████████████████████████████   ] 92% 
epoch: 14/14  acc: 100.00  loss: 0.001 
[████████████████████████████████████████] 100% 

(array([[ 1.38503523, -0.51962709]]), array([[-1.31788536,  1.49334281],
        [-0.10027775, -1.39507145]], dtype=float128))

In [18]:
### from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np

from mlfromscratch.deep_learning import NeuralNetwork
from mlfromscratch.utils import train_test_split, to_categorical, normalize, Plot
from mlfromscratch.utils import get_random_subsets, shuffle_data, accuracy_score
from mlfromscratch.deep_learning.optimizers import StochasticGradientDescent, Adam, RMSprop, Adagrad, Adadelta
from mlfromscratch.deep_learning.loss_functions import CrossEntropy
from mlfromscratch.utils.misc import bar_widgets
from mlfromscratch.deep_learning.layers import RNN, Activation


def main():

    optimizer = Adam()
#     optimizer = StochasticGradientDescent()

    def gen_mult_ser(nums):
        """ Method which generates multiplication series """
        X = np.zeros([nums, 10, 61], dtype=float)
        y = np.zeros([nums, 10, 61], dtype=float)
        for i in range(nums):
            start = np.random.randint(2, 7)
            mult_ser = np.linspace(start, start*10, num=10, dtype=int)
            X[i] = to_categorical(mult_ser, n_col=61)
            y[i] = np.roll(X[i], -1, axis=0)
        y[:, -1, 1] = 1 # Mark endpoint as 1
        return X, y


    def gen_num_seq(nums):
        """ Method which generates sequence of numbers """
        X = np.zeros([nums, 10, 20], dtype=float)
        y = np.zeros([nums, 10, 20], dtype=float)
        for i in range(nums):
            start = np.random.randint(0, 10)
            num_seq = np.arange(start, start+10)
            X[i] = to_categorical(num_seq, n_col=20)
            y[i] = np.roll(X[i], -1, axis=0)
        y[:, -1, 1] = 1 # Mark endpoint as 1
        return X, y

    X, y = gen_mult_ser(3000)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

    # Model definition
    clf = NeuralNetwork(optimizer=optimizer,
                        loss=CrossEntropy)
    clf.add(RNN(10, activation="tanh", bptt_trunc=5, input_shape=(10, 61)))
    clf.add(Activation('softmax'))
    clf.summary("RNN")

    # Print a problem instance and the correct solution
    tmp_X = np.argmax(X_train[0], axis=1)
    tmp_y = np.argmax(y_train[0], axis=1)
    print ("Number Series Problem:")
    print ("X = [" + " ".join(tmp_X.astype("str")) + "]")
    print ("y = [" + " ".join(tmp_y.astype("str")) + "]")
    print ()

    train_err, _ = clf.fit(X_train, y_train, n_epochs=500, batch_size=512)

    # Predict labels of the test data
    y_pred = np.argmax(clf.predict(X_test), axis=2)
    y_test = np.argmax(y_test, axis=2)

    print ()
    print ("Results:")
    for i in range(5):
        # Print a problem instance and the correct solution
        tmp_X = np.argmax(X_test[i], axis=1)
        tmp_y1 = y_test[i]
        tmp_y2 = y_pred[i]
        print ("X      = [" + " ".join(tmp_X.astype("str")) + "]")
        print ("y_true = [" + " ".join(tmp_y1.astype("str")) + "]")
        print ("y_pred = [" + " ".join(tmp_y2.astype("str")) + "]")
        print ()
    
    accuracy = np.mean(accuracy_score(y_test, y_pred))
    print ("Accuracy:", accuracy)

    training = plt.plot(range(500), train_err, label="Training Error")
    plt.title("Error Plot")
    plt.ylabel('Training Error')
    plt.xlabel('Iterations')
    plt.show()

main()

+-----+
| RNN |
+-----+
Input Shape: (10, 61)
+----------------------+------------+--------------+
| Layer Type           | Parameters | Output Shape |
+----------------------+------------+--------------+
| RNN                  | 1320       | (10, 61)     |
| Activation (Softmax) | 0          | (10, 61)     |
+----------------------+------------+--------------+
Total Parameters: 1320

Number Series Problem:
X = [4 8 12 16 20 24 28 32 36 40]
y = [8 12 16 20 24 28 32 36 40 1]

loss 0.09048720433257987
(512, 61)
(10, 61) no
loss 0.09044689560591289
(512, 61)
(10, 61) no
loss 0.0903972973036759
(512, 61)
(10, 61) no


loss 0.09033328370452352
(264, 61)
(10, 61) no
loss 0.090263718248511
(512, 61)
(10, 61) no
loss 0.09019414932633613
(512, 61)
(10, 61) no
loss 0.0901246764183648
(512, 61)
(10, 61) no
loss 0.09004394854217194
(264, 61)
(10, 61) no
loss 0.08996327567058172
(512, 61)
(10, 61) no
loss 0.08988666647311848
(512, 61)
(10, 61) no
loss 0.08981176237866573
(512, 61)
(10, 61) no
loss 0.08972343768781156
(264, 61)
(10, 61) no
loss 0.08963858109114234
(512, 61)
(10, 61) no
loss 0.08955961835702578
(512, 61)
(10, 61) no
loss 0.08948240394023542
(512, 61)
(10, 61) no
loss 0.08938787091294614
(264, 61)
(10, 61) no
loss 0.08929957370377885
(512, 61)
(10, 61) no


loss 0.089218635904666
(512, 61)
(10, 61) no
loss 0.0891390124280808
(512, 61)
(10, 61) no
loss 0.08903737996073179
(264, 61)
(10, 61) no
loss 0.0889445749193523
(512, 61)
(10, 61) no
loss 0.08886077396715944
(512, 61)
(10, 61) no


KeyboardInterrupt: 

In [ ]:
X_train[100].shape